In [12]:
import os
import numpy as np
import pandas as pd

INPUT_DIR = "./data"

def read_csv(filename):
    df = pd.read_csv(os.path.join(INPUT_DIR, filename))
    first_close = df["close"][0]
    net_values = df["close"] / first_close
    df["net_values"] = net_values
    return df

df_pfyh = read_csv("pfyh.csv")
df_zjls = read_csv("zjls.csv")


In [13]:
# Compute rate-of-return
def compute_annual_profit(days, net_value, days_per_year=245):
    assert days > 0, "Days must be positive!"
    years = days / days_per_year
    annual_profit = pow(net_value, 1 / years) - 1
    annual_profit = round(annual_profit * 100, 2)
    return annual_profit

annual_profit_pfyh = compute_annual_profit(len(df_pfyh), df_pfyh["net_values"][df_pfyh.index[-1]])
annual_profit_zjls = compute_annual_profit(len(df_zjls), df_zjls["net_values"][df_zjls.index[-1]])

print("浦发银行年化收益率：", annual_profit_pfyh, "%")
print("浙江龙盛年化收益率：", annual_profit_zjls, "%")
print("策略（简单平均）年化收益率：", round((annual_profit_pfyh + annual_profit_zjls) / 2, 2), "%")

浦发银行年化收益率： -1.9 %
浙江龙盛年化收益率： 9.52 %
策略（简单平均）年化收益率： 3.81 %


In [14]:
# Compute max-draw-down
def compute_drawdown(net_values):
    max_drawdown = 0
    index = 0
    for net_value in net_values:
        for sub_net_value in net_values[index: ]:
            drawdown = 1 - sub_net_value / net_value
            max_drawdown = max(max_drawdown, drawdown)
        index += 1
    return max_drawdown

max_drawdown_pfyh = compute_drawdown(df_pfyh["net_values"])
max_drawdown_zjls = compute_drawdown(df_zjls["net_values"])
max_drawdown_pair = compute_drawdown((df_zjls["net_values"] + df_pfyh["net_values"]) / 2)

print("浦发银行最大回撤：", round(max_drawdown_pfyh * 100, 2), "%")
print("浙江龙盛最大回撤：", round(max_drawdown_zjls * 100, 2), "%")
print("策略（简单平均）最大回撤：", round(max_drawdown_pair * 100, 2), "%")


浦发银行最大回撤： 32.72 %
浙江龙盛最大回撤： 53.2 %
策略（简单平均）最大回撤： 42.24 %


In [18]:
# Compute sharpe ratio
def compute_sharpe_ratio(net_values, days_per_year=245):
    net_values = list(net_values)
    trading_days = len(net_values)
    profit_df = pd.DataFrame(columns={"profit"})
    profit_df.loc[0] = {"profit": round((net_values[0] - 1) * 100, 2)}
    for index in range(1, trading_days):
        profit_rate = (net_values[index] - net_values[index - 1]) / net_values[index - 1]
        profit_rate = round(profit_rate * 100, 2)
        profit_df.loc[index] = {"profit": profit_rate}
    
    profit_std = pow(profit_df["profit"].var(), 0.5)

    annual_profit = compute_annual_profit(trading_days, net_values[-1])

    df_gzzs = read_csv("gzzs.csv")
    norisk_profit = compute_annual_profit(trading_days, df_gzzs["net_values"][df_gzzs.index[-1]])
    
    sharpe_ratio = (annual_profit - norisk_profit) / (profit_std * pow(days_per_year, 0.5))
    return sharpe_ratio

sharpe_ratio_pfyh = compute_sharpe_ratio(df_pfyh["net_values"])
sharpe_ratio_zjls = compute_sharpe_ratio(df_zjls["net_values"])
sharpe_ratio_pair = compute_sharpe_ratio((df_pfyh["net_values"] + df_zjls["net_values"]) / 2)

print("浦发银行夏普比率：", round(sharpe_ratio_pfyh, 3))
print("浙江龙盛夏普比率：", round(sharpe_ratio_zjls, 3))
print("策略（简单平均）夏普比率：", round(sharpe_ratio_pair, 3))

浦发银行夏普比率： -0.272
浙江龙盛夏普比率： 0.168
策略（简单平均）夏普比率： 0.036
